In [1]:
import os
import glob
import pandas as pd
from datetime import datetime
import locale
import re

In [11]:
class File_manager:

    # 생성자
    def __init__(self, fund_code):
        # Ensure the fund code is a string
        self.fund_code = str(fund_code)
        self.df = None
        self.folder_path = '/Users/ihan-il/Desktop/Python Folder/FUND_CLASS/캡스톤데이터2'

    # 파일 이름 가져오기
    def scan_files_including_regex(self, file_folder, regex, option='path'):
        with os.scandir(file_folder) as files:
            lst = [file.name for file in files if re.findall(regex, file.name)]
        mapping = {
            'name': lst,
            'path': [os.path.join(file_folder, file_name) for file_name in lst]
        }
        return mapping[option]
        
    # 정규 표현식을 사용하여 날짜와 시간 부분을 추출하는 함수
    def extract_datetime(self, file_name):
        # 정규 표현식으로 날짜와 시간 추출 (시간은 옵션)
        match = re.search(r'save(\d{4})(\d{2})(\d{2})(\d{4})?', file_name)
        if match:
            # 연, 월, 일은 항상 있으며, 시간이 없는 경우 0000으로 설정
            date_part = match.group(1) + match.group(2) + match.group(3)
            time_part = match.group(4) if match.group(4) else "0000"
            return datetime.strptime(date_part + time_part, '%Y%m%d%H%M')
        else:
            return None

    # 최근의 파일 가져오기 
    def get_latest_file(self, file_code):
        file_code = str(file_code)
        file_list = self.scan_files_including_regex(self.folder_path, regex = f'menu{file_code}-code{self.fund_code}')
        sorted_file_list = sorted(file_list, key=self.extract_datetime, reverse=True)

        if sorted_file_list:
            return sorted_file_list[0]
        else:
            raise FileNotFoundError(f"No file found with file code {file_code} and fund code {self.fund_code}")

    
    def get_file_as_df_with_filecode(self, file_code, merge_header=False):
        # 파일 리스트를 수정 시간 기준으로 내림차순 정렬하여, 가장 최근에 수정된 파일이 리스트의 첫 번째에 오도록 합니다.
        latest_file = self.get_latest_file(file_code)
        
        # merge_header가 참일 경우
        if merge_header:
            # Read CSV without headers
            df = pd.read_csv(latest_file, header=None)
            # Get the first two rows to create the header
            headers = self.create_header(df.iloc[0], df.iloc[1])
            # Assign the new headers to the DataFrame and drop the first two rows
            df.columns = headers
            df = df.drop([0, 1])
        else:
            # Read CSV with default headers
            df = pd.read_csv(latest_file)
        return df
    
    # m2205를 위한 데이터 프레임 가지고 오는 법 
    def get_file_as_df_with_filepath(self, file_path, merge_header=False):
        # 파일 리스트를 수정 시간 기준으로 내림차순 정렬하여, 가장 최근에 수정된 파일이 리스트의 첫 번째에 오도록 합니다.
        
        # merge_header가 참일 경우
        if merge_header:
            # Read CSV without headers
            df = pd.read_csv(file_path, header=None)
            # Get the first two rows to create the header
            headers = self.create_header(df.iloc[0], df.iloc[1])
            # Assign the new headers to the DataFrame and drop the first two rows
            df.columns = headers
            df = df.drop([0, 1])
        else:
            # Read CSV with default headers
            df = pd.read_csv(file_path)
        return df


    # 데이터프레임 헤더 처리
    def create_header(self, row1, row2):
        # Combine row1 and row2 according to the specified rules
        header = []
        for col1, col2 in zip(row1, row2):
            # 개행 문자를 빈 문자열로 바꿈
            clean_col1 = col1.replace('\n', '') if isinstance(col1, str) else col1
            clean_col2 = col2.replace('\n', '') if isinstance(col2, str) else col2
            
            # If the second row has a value, use it, otherwise use the first row's value
            header.append(clean_col2 if pd.notnull(clean_col2) and clean_col2 != '' else clean_col1)
        return header
    
    def make_unique_column_names(self, df):
        # 컬럼 이름에 중복이 있는 경우, 고유한 이름을 생성합니다.
        cols = df.columns
        unique_cols = {}
        new_cols = []
        
        for col in cols:
            if col in unique_cols:
                unique_cols[col] += 1
                new_col = f"{col}.{unique_cols[col]}"
            else:
                unique_cols[col] = 1
                new_col = col
            new_cols.append(new_col)
        
        df.columns = new_cols
    
    def save_df(self):

        # Define the folder path for 'save_file' directory
        folder_path = 'save_file'

        # Check if folder exists, if not, create it
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        # Get the current timestamp
        save_timestamp = datetime.now().strftime('%Y%m%d_%H%M')

        # Define the file name with current timestamp
        file_name = f'menu_2305_save{save_timestamp}.csv'

        # Complete file path
        file_path = os.path.join(folder_path, file_name)

        # Save the dataframe to CSV
        self.df.to_csv(file_path, index=False)

        print(f"File saved to {file_path}")

# 그 외 공통 메서드들
class Etc_df_Processor:
    def __init__(self, df):
        self.df = df
        
    def convert_number_format(self, column_name):
        self.df[column_name] = self.df[column_name].apply(self._convert_number_format_helper).astype(float)

    def _convert_number_format_helper(self, s):
        return float(s.replace(',', '')) if isinstance(s, str) else s

    def replace_values_less_than_or_equal_to_zero(self, *column_names):
        for column_name in column_names:
            self.df.loc[self.df[column_name] <= 0, column_name] = None
            self.df[column_name].ffill(inplace=True)

In [2]:
class M8186:
    def __init__(self, fund_code, input_date, menu_code = '8186'):
        self.fund_code = fund_code
        self.menu_code = menu_code
        self.input_date = input_date
        self.df_ref = self.get_df_ref()
        
        self.fund_name =self.get_fund_name()
        self.manager = self.get_manager()
        self.inception_date = self.get_inception_date() 
        self.inception_price = self.get_inception_price()
        self.net_asset_value = self.get_net_asset_value()
        self.reference_price = self.get_reference_price()
        self.cum_return = self.get_cum_return()
        
        
    def open_df_raw(self):
        lst = scan_files_including_regex('./캡스톤데이터', f'menu{self.menu_code}-code{self.fund_code}')
        lst = sorted(lst, reverse=True)
        file_path = lst[0]
        df = pd.read_csv(file_path)
        return df
    
    def get_df_ref(self):
        df = self.open_df_raw()
        df = df[['일자', '펀드명', '운용역', '설정일', '설정액', '순자산', '수정기준가']]
        df['수정기준가'] = df['수정기준가'].apply(lambda x: float(x.replace(',', '')) if isinstance(x, str) else x)
        return df

    def get_fund_name(self):
        df = self.df_ref
        df = df[df['일자']==self.input_date][['펀드명']]      
        fund_name = df['펀드명'].iloc[0] 
        return fund_name    
    
    def get_manager(self):
        df = self.df_ref
        df = df[df['일자']==self.input_date][['운용역']]      
        manager = df['운용역'].iloc[0] 
        return manager   
    
    def get_inception_date(self):
        df = self.df_ref
        df = df[df['일자']==self.input_date][['설정일']]      
        inception_date = df['설정일'].iloc[0]    
        return inception_date
    
    def get_inception_price(self):
        df = self.df_ref
        df = df[df['일자']==self.input_date][['설정액']]      
        inception_price = df['설정액'].iloc[0]    
        return inception_price   
    
    def get_net_asset_value(self):
        df = self.df_ref
        df = df[df['일자']==self.input_date][['순자산']]      
        net_asset_value = df['순자산'].iloc[0]    
        return net_asset_value        

    def get_reference_price(self):
        df = self.df_ref
        df = df[df['일자']==self.input_date][['수정기준가']]      
        reference_price = df['수정기준가'].iloc[0]    
        return reference_price        
    
    def get_cum_return(self):
        df = self.df_ref
        first_price = df['수정기준가'].iloc[0]
        df = df[df['일자']==self.input_date][['수정기준가']]
        last_price = df['수정기준가'].iloc[0]
        cum_return = round((last_price-first_price)/first_price * 100, 2)
        return cum_return
        
    def get_info(self):
        info_data = {
            '항목': ['펀드명', '운용역', '운용개시일', '계약금액', '순자산금액', '기준가격', '누적수익률'],
            '값': [self.fund_name, self.manager, self.inception_date, self.inception_price, 
                   self.net_asset_value, self.reference_price, self.cum_return]
        }

        info_df = pd.DataFrame(info_data)
        return info_df
        
        
        
        
class M2820:
    def __init__(self, fund_code,start_date, end_date, menu_code = '2820'):
        self.fund_code = fund_code
        self.start_date = start_date
        self.end_date = end_date
        self.menu_code = menu_code

        
def scan_files_including_regex(file_folder, regex, option='path'):
    with os.scandir(file_folder) as files:
        lst = [file.name for file in files if re.findall(regex, file.name)]
    mapping = {
        'name': lst,
        'path': [os.path.join(file_folder, file_name) for file_name in lst]
    }
    return mapping[option]

In [3]:
lst = scan_files_including_regex('./캡스톤데이터', 'menu8186-code100004')
lst = sorted(lst, reverse=True)
lst

['./캡스톤데이터/menu8186-code100004-save202311021348.csv']

In [5]:
m = M8186(fund_code='A00001', input_date='2023-09-30')
info = m.get_info()
info

,항목,값
0,펀드명,라이프 ESG형 1호
1,운용역,강대권
2,운용개시일,2023-05-11
3,계약금액,"10,000,000,000"
4,순자산금액,"10,785,529,144"
5,기준가격,1078.55
6,누적수익률,7.89


In [6]:
def create_dataframe(career_summary = None, education_certifications = None):
    
    career_summary = (
    "한국투자밸류자산운용 주식운용본부(주식운용), 2007.12 ~ 2014.02\n"
    "유경PSG자산운용 CIO(주식운용), 2014.02 ~ 2021.03\n"
    "라이프자산운용 공동대표(주식운용), 2021.06 ~ 현재"
    )
    education_certifications = (
        "서울대 경제학 학/석사\n"
        "투자자산운용사\n"
        "부동산투자자산운용사"
    )
    # 멀티인덱스 생성
    columns = pd.MultiIndex.from_tuples(
        [('성명', ''), ('직위', ''), ('생년월일', ''), ('연락처', ''), ('주요경력', '전,현직장의 근무부서 및 기간, 재직시 담당 업무'), ('주요경력', '학위 및 업무관련 자격증 취득 현황')]
    )

    # 멀티인덱스를 사용하는 데이터프레임 생성
    df = pd.DataFrame(columns=columns)

    # 첫 번째 행에 값을 입력, 주요경력 부분에는 변수 사용
    df.loc[0] = ["강대권", "대표이사", "1980.11.24", "02-6105-6832", career_summary, education_certifications]

    # 데이터프레임 전치하여 반환
    return df.T

In [7]:
Investment_Professional_df = create_dataframe()
Investment_Professional_df

0
성명                                                                              강대권
직위                                                                             대표이사
생년월일                                                                     1980.11.24
연락처                                                                    02-6105-6832
주요경력 전,현직장의 근무부서 및 기간, 재직시 담당 업무  한국투자밸류자산운용 주식운용본부(주식운용), 2007.12 ~ 2014.02\n유경...
     학위 및 업무관련 자격증 취득 현황                          서울대 경제학 학/석사\n투자자산운용사\n부동산투자자산운용사

In [9]:
class ComplianceStatus:
    def __init__(self, fund_code, start_date=None, end_date=None):
        self.fund_code = fund_code
        self.file_manager = File_manager(fund_code)
        self.start_date = start_date
        self.end_date = end_date
        self.date_column = '일자'

    def open_df_8186_raw(self):
        df = self.file_manager.get_file_as_df_with_filecode('8186')
        df['순자산'] = df['순자산'].str.replace(',', '').astype(float)
        return df[['일자', '순자산']]

    def open_df2820_raw(self):
        df = self.file_manager.get_file_as_df_with_filecode('2820')
        df = df[['일자', '매매구분', '수수료', '취득액']]
        df['수수료'] = df['수수료'].str.replace(',', '').astype(float)
        df['취득액'] = df['취득액'].str.replace(',', '').astype(float)
        return df.groupby(['일자', '매매구분']).agg({'수수료': 'sum', '취득액': 'sum'}).reset_index()

    def filter_by_date_range(self, df):
        df[self.date_column] = pd.to_datetime(df[self.date_column])
        if self.start_date and self.end_date:
            df = df[(df[self.date_column] >= self.start_date) & (df[self.date_column] <= self.end_date)]
        elif self.start_date:
            df = df[df[self.date_column] >= self.start_date]
        elif self.end_date:
            df = df[df[self.date_column] <= self.end_date]
        return df

    def merge_data(self):
        df_8186 = self.filter_by_date_range(self.open_df_8186_raw())
        df_2820 = self.filter_by_date_range(self.open_df2820_raw())
        merged_df = pd.merge(df_2820, df_8186, on='일자', how='inner')
        merged_df['일자'] = pd.to_datetime(merged_df['일자'])
        return merged_df

    def calculate_monthly_turnover(self, merged_df):
        monthly_turnovers = {}
        for period in merged_df['일자'].dt.to_period('M').unique():
            month_df = merged_df[merged_df['일자'].dt.to_period('M') == period]
            # 주식장내매도와 ETF장내매도의 취득액만 고려
            total_acquisition = month_df[month_df['매매구분'].isin(['주식장내매도', 'ETF장내매도'])]['취득액'].sum()
            average_net_asset = month_df['순자산'].mean()
            monthly_turnovers[period.strftime('%Y-%m')] = total_acquisition / average_net_asset if average_net_asset else 0
        return monthly_turnovers
    
    def calculate_monthly_fee(self, merged_df):
        monthly_fees = {}
        for period in merged_df['일자'].dt.to_period('M').unique():
            month_df = merged_df[merged_df['일자'].dt.to_period('M') == period]
            fee = month_df['수수료'].sum()
            monthly_fees[period.strftime('%Y-%m')] = fee
        return monthly_fees

    def get_df_performance(self):
        merged_df = self.merge_data()
        monthly_turnovers = self.calculate_monthly_turnover(merged_df)
        monthly_fees = self.calculate_monthly_fee(merged_df)

        # 월간 값과 누적 값 계산
        last_turnover = list(monthly_turnovers.values())[-1] if monthly_turnovers else 0
        last_fee = list(monthly_fees.values())[-1] if monthly_fees else 0
        total_turnover = sum(monthly_turnovers.values())
        total_fee = sum(monthly_fees.values())
        last_turnover = round(last_turnover*100,2)
        total_turnover = round(total_turnover*100,2)

        # 데이터 프레임 생성
#         performance_df = pd.DataFrame({
#             '월간': [last_turnover, last_fee],
#             '누적': [total_turnover, total_fee]
#         }, index=['매매회전율', '매매수수료'])
        turnover_rate = f"{last_turnover}/{total_turnover}"
        transaction_fee = f"{last_fee}/{total_fee}"
        no_records = '해당내역없음'
        performance_df = pd.DataFrame({
            'header': [no_records, no_records,turnover_rate, transaction_fee,no_records,no_records],
        }, index=['운용대상 제한', '편입비 제한', '매매회전율(월간/누적)', '매매수수료(월간/누적)', '종목손절매', '투자일임자산 및 고유자산과의 중복거래 내역'])        
        
        return performance_df

In [12]:
compliance_calculator = ComplianceStatus('A00001',start_date= '2023-05-11', end_date= '2023-09-30')
compliance_calculator.get_df_performance()

,header
운용대상 제한,해당내역없음
편입비 제한,해당내역없음
매매회전율(월간/누적),15.01/77.14
매매수수료(월간/누적),3413800.0/23251600.0
종목손절매,해당내역없음
투자일임자산 및 고유자산과의 중복거래 내역,해당내역없음
